# Dataset Overview
I will be using two Chesterton texts, Father Brown and The Man Who Was Thursday, along with two Austen texts, Emma and Persuasion.

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
#Search Gutenberg Corpus and find texts
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


# Cleaning, Processing and Parsing

In [33]:
#Load the raw text
emma = gutenberg.raw('austen-emma.txt')
persuasion = gutenberg.raw('austen-persuasion.txt')
brown = gutenberg.raw('chesterton-brown.txt')
thursday = gutenberg.raw('chesterton-thursday.txt')

** Inspect raw files first

In [34]:
#Print first 500 characters of each, followed by 'END'
print(emma[0:500])
print('\n\n END')

print(persuasion[0:500])
print('\n\n END')

print(brown[0:500])
print('\n\n END')

print(thursday[0:500])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t


 END
[Persuasion by Jane Austen 1818]


Chapter 1


Sir Walter Elliot, of Kellynch Hall, in Somersetshire, was a man who,
for his own amusement, never took up any book but the Baronetage;
there he found occupation for an idle hour, and consolation in a
distressed one; there his faculties were roused into admiration and
respect, by contemplating the limited remnant of the earliest patents;
there any unwelcome sensations, arising from domestic affairs
changed naturally into pity and contempt as

** Clean and Process the Texts

In [36]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub("\d+", "", text)
    text = re.sub(r'Chapter \d+', '', text)
    text = re.sub(r'VOLUME \w+', '', text)
    text = re.sub(r'CHAPTER \w+', '', text)
    text = ' '.join(text.split())
    return text

In [37]:
# Run text cleaning utility on our texts
emma = text_cleaner(emma)
persuasion = text_cleaner(persuasion)
brown = text_cleaner(brown)
thursday = text_cleaner(thursday)

** Parse the Texts

In [40]:
# Parse the cleaned texts
nlp = spacy.load('en')

emma_doc = nlp(emma)
persuasion_doc = nlp(persuasion)
brown_doc = nlp(brown)
thursday_doc = nlp(thursday)

In [41]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
brown_sents = [[sent, "Chesterton"] for sent in brown_doc.sents]
thursday_sents = [[sent, "Chesterton"] for sent in thursday_doc.sents]

print(len(emma_sents))
print(len(persuasion_sents))
print(len(brown_sents))
print(len(thursday_sents))

8905
3656
3716
3490


In [44]:
#For computational purposes, reduce length of each set of sentences to 500
emma_sents = emma_sents[0:500]
persuasion_sents = persuasion_sents[0:500]
brown_sents = brown_sents[0:500]
thursday_sents = thursday_sents[0:500]

print(len(emma_sents))
print(len(persuasion_sents))
print(len(brown_sents))
print(len(thursday_sents))

500
500
500
500


In [49]:
# Combine the sentences from the four texts into one data frame.
sentences = pd.DataFrame(emma_sents + persuasion_sents + brown_sents + thursday_sents)

#Confirm this worked
display(sentences.head(5))

#Confirm we only have two authors across four texts
print(sentences.iloc[:, 1].unique())

,0,1
0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen
1,"(She, was, the, youngest, of, the, two, daught...",Austen
2,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,"(Between, _, them)",Austen


['Austen' 'Chesterton']


# Create DataFrame with BoW and TF-IDF Features

In [45]:
# Utility function to create a list of the 1000 most common words

def bag_of_words(text):
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
        
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]

In [50]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.

def bow_features(sentences, common_words):
    
    # Create column headers for sentence text and source (author) and initialize to 0
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [51]:
# Set up the bags for each text
emmawords = bag_of_words(emma_doc)
persuasionwords = bag_of_words(persuasion_doc)
brownwords = bag_of_words(brown_doc)
thursdaywords = bag_of_words(thursday_doc)

# Combine bags to create a set of unique words.
common_words = set(emmawords + persuasionwords + brownwords + thursdaywords)

** Create BoW Features

In [52]:
# Create our data frame with features
word_counts = bow_features(sentences, common_words)
word_counts.head(10)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500


,assistance,somewhat,favourite,thorough,goddard,pleasing,steam,series,shirley,conversation,...,attachment,acknowledge,convey,heaven,walter,instead,daylight,tenderness,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(She, was, the, youngest, of, the, two, daught...",Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Between, _, them)",Austen
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,(_),Austen
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(it, was, more, the, intimacy, of, sisters, .)",Austen
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Even, before, Miss, Taylor, had, ceased, to, ...",Austen
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, real, evils, ,, indeed, ,, of, Emma, 's,...",Austen
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, danger, ,, however, ,, was, at, present,...",Austen


In [ ]:
#Counting Parts of Speech

In [ ]:
#Counting Entities

In [ ]:
#Count of Words in Sentence

** Create TF-IDF Features

In [ ]:
#Create vectorizer model in order to get tf-idf for each sentence

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer -- will be done to source_text column in our dataframe since we already parsed sentences
emma_paras_tfidf=vectorizer.fit_transform(emma_paras)
print(emma_paras_tfidf)
print(emma_paras_tfidf.shape)

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(emma_paras_tfidf, test_size=0.4, random_state=0)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features